# Credit Card Fraud Detection

In [ ]:
# Carrega as libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Load Dataset

In [ ]:
# diretorio de trabalho
df = pd.read_csv('../input/creditcard.csv')
df.head()

In [ ]:
df.shape

## Análise das Variáveis

### Análise da Variável Resposta - Classes 0 e 1
- 0 = Transação Legítima
- 1 = Transação Fraudulenta

In [ ]:
# value_counts == table() do R
class_count = pd.value_counts(df['Class'])
print(class_count)

# Plota as classes
ax = sns.countplot('Class', data = df)
for p in ax.patches:
    ax.annotate('{:.2f}%'.format(100*p.get_height()/len(df['Class'])), (p.get_x() + 0.35, p.get_height() + 3000))

### Análise das Variáveis de Interesse
As variáveis apresentadas estão de forma anônimas, exceto a varável **Time** (medida em segundos da transação a partir da primeira) e **Amount** (valor da transação). As demais 28 variáveis são numéricas.

___
- **Variável Time**

In [ ]:
# Para analisar a variável Time, vou reajustá-la para observar em Horas
df['Time_Hours'] = df['Time']/3600

In [ ]:
df['Time_Hours'].tail()

Como podemos observar acima, o dataset contém infos de 2 dias (48h)

Obs: Mais para frente vamos observar melhor o comportamento das variáveis de acordo com o tempo

___
- **Variável Amount**

In [ ]:
plt.figure(figsize=(12,3))
sns.distplot(df['Amount'], kde = False);

Observamos acima que as transações de um valor de até 1000 representam a maior parte dos dados.

In [ ]:
# Vamos dar um zoom nessas transações
plt.figure(figsize=(20,5))
plt.subplot().set_xlim(-10, 1000)   # seleciona o zoom do valor 0 até 2000
sns.distplot(df[df['Amount']<=1000].Amount);

Mais ainda, estão altamente concentradas entre 0 e 200
___
Para resolver isso vai ser necessário normalizar os dados. Existem diversas formas de realizar isso, aqui vou usar o StandardScaler do sklearn.

In [ ]:
# StandardScaler deixa a média = 0, e dp = 1
from sklearn.preprocessing import StandardScaler

df['Amount_Norm'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1))
print('Média: ' + str(np.round(np.mean(df['Amount_Norm']))))
print('Var: ' + str(np.round(np.var(df['Amount_Norm']))))

Uma outra maneira de tratar esses valores é criando ranges, assim transformando em variável categorica.

___
- **Variáveis Descaracterizadas (V1 - V28)**

Aqui vamos observar a distribuição de todas as variáveis, assim tendo uma visão geral de tudo.

Como vimos nos plots, as distribuições estão parecidas.

In [ ]:
######
# Vou plotar as distribuições de todas as variáveis
import itertools

# Define o espaço para os 28 plots
fig, axes = plt.subplots(7, 4, sharex=True, sharey=True, figsize=(20,10))

var = 1   # vai ser usado para o nome das variaveis V1 - V28
# loop para plotar os graficos
for i, j in itertools.product(range(7), range(4)):
    axes[i,j].set(xlim=(-10, 10), ylim=(0, 1.2))   # ajusta a escala das dimensoes x e y
    sns.distplot(df['V' + str(var)], ax=axes[i,j])
    var = var + 1

## Modelos de Regressão Logística

In [ ]:
# Cria funcao para plotar a matriz de confusao
def PlotCM(y_test, pred, plot = True):
    
    from sklearn.metrics import confusion_matrix, classification_report
    
    # Cria CM e CM normalizada
    cm_matrix = confusion_matrix(y_test, pred)
    cm_matrix_norm = cm_matrix / cm_matrix.astype(np.float).sum(axis=1)
    
    fig = plt.figure(figsize=(12, 3))    
    # Plota CM
    ax = fig.add_subplot(1,2,1)
    sns.heatmap(cm_matrix, cmap='coolwarm_r', linewidths=0.5, annot=True, fmt='g', ax=ax)
    plt.title('Confusion Matrix')
    plt.ylabel('Real Classes')
    plt.xlabel('Predicted Classes')
    
    # Plota CM Normalizada
    # variavel para controlar se plota o grafico ou somente gera os valores
    if (plot == True):
        ax = fig.add_subplot(1,2,2)
        sns.heatmap(cm_matrix_norm, cmap='coolwarm_r', linewidths=0.5, annot=True, fmt='g', ax=ax)
        plt.title('Normalized Confusion Matrix')
        plt.ylabel('Real Classes')
        plt.xlabel('Predicted Classes')
        plt.show()
    
    print('---Classification Report---')
    TP = cm_matrix[1,1]
    FN = cm_matrix[1,0]
    FP = cm_matrix[0,1]
    TN = cm_matrix[0,0]
    T = TP+FN+FP+TN
    
    print('Accuracy =      {:.3f}'.format((TP+TN)/T))
    print('Specificity =   {:.3f}'.format(TN/(TN+FP)))
    print('Precision =     {:.3f}'.format(TP/(TP+FP)))
    print('Recall (TPR) =  {:.3f}'.format(TP/(TP+FN)))
    print('Fallout (FPR) = {:.3e}'.format(FP/(FP+TN)))
    print('F1 Score =      {:.3f}'.format(2*TP/(2*TP+FP+FN)))
    print('\n')

In [ ]:
# Cria Treino e Teste
def cross_val_model(X, y, model, n_splits=3, CM = True):
    # Cria os 
    
    from sklearn.model_selection import StratifiedKFold
    
    X = np.array(X)
    y = np.array(y)

    folds = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123).split(X, y))

    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = X[train_idx]
        y_train = y[train_idx]
        X_test = X[test_idx]
        y_test = y[test_idx]

        print ("Fit %s fold %d" % (str(model).split('(')[0], j+1))
        model.fit(X_train, y_train)
        
        pred = model.predict(X_test)
        
        PlotCM(y_test, pred, CM)

### Exemplo com CV

In [ ]:
# Separa a base em treino/validação e teste
from sklearn.model_selection import train_test_split

# Cria X e y
var_x = ['Time','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14',
         'V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']
var_y = ['Class']

X = np.array(df.loc[:,var_x])
y = np.array(df.loc[:,var_y])

# Separa em treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()

# Treina o modelo
cross_val_model(X_train, y_train, lr_model, n_splits=3, CM = True)

### Feature Selection (Recursive feature elimination)
Mais especificamente o Feature ranking with recursive feature elimination and cross-validation (RFECV).

Além de técnicas de feature selection, as análises exploratórias são fundamentais para conseguir escolher as variáveis finais no modelo, e ter insights sobre algum detalhe, transformação ou criação das variáveis.

In [ ]:
from sklearn.feature_selection import RFECV
# Create the RFE object and compute a cross-validated score.
# The "accuracy" scoring is proportional to the number of correct classifications
rfecv = RFECV(estimator=LogisticRegression(), step=1, cv=5, scoring='precision')
rfecv.fit(X_train, y_train)

In [ ]:
print("Optimal number of features: %d" % rfecv.n_features_)
print('Selected features: %s' % list(df.loc[:,var_x].columns[rfecv.support_]))

# Plot number of features VS. cross-validation scores
plt.figure(figsize=(10,6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
# Atualiza as bases com as variáveis selecionadas
X_train = X_train[:,rfecv.support_]
X_test = X_test[:,rfecv.support_]

In [ ]:
# Treina o modelo
cross_val_model(X_train, y_train, lr_model, n_splits=3, CM = True)

Já conseguimos ver que o modelo consegue melhorar a performance, principalemente observando a medida Precision.
Além disso podemos melhor ainda mais nosso modelo, repetindo esses passos, explorando outras variáveis e com insights das análises e gráficos.

In [ ]:
# Plota a correlação das variáveis selecionadas e variável resposta
selected_features = np.array(df.loc[:,np.array(var_x)[rfecv.support_]].columns)
X_corr = df[np.concatenate([selected_features, np.array(var_y)])]

plt.subplots(figsize=(20, 5))
sns.heatmap(X_corr.corr(), annot=True, cmap="RdYlGn")
plt.show()

### Undersampling

Aqui vou utilizar a técnica de undersampling para tratar o desbalanceamento das classes. O undersampling 'retira' observações da classe predominante para igualar a classe pouco presente.

Estou utilizando essa técnica, pois nesse problema é mais fácil dizer que a transação não é fraude, então tirar observações dessa classe não afetará na identificação da mesma. Já a classe de 'fraude' é mais sensível e difícil de identificar, então um oversampling poderia atrapalhar na criação do modelo, incluindo uma maior possibilidade de overfitting.

#### Preparação dos Dados
Prepara os dataframe para tirar observações de classe 0 com proporções 3:1 e 2:1.

**OBS: Essa análise é apenas para exemplo. Para verificar a eficácia da técnica é necessário treinar e testar o modelo de uma forma mais exaustiva, pois muitos dados foram deixados de fora.
Além disso, o df é muito desbalanceado, então proporções maiores já podem trazer resultados bons.**

In [ ]:
######
# Vou criar 2 datasets, o primeiro com classes 3:1 e 2:1
id_1 = np.where(y_train==1)[0] # ids das linhas com variavel resposta 1
id_0 = np.where(y_train==0)[0] # ids das linhas com variavel resposta 0
n_class_1 = len(id_1)  # numero de linhas com variavel resposta 1

# 100:1
id_subset = np.concatenate([np.random.choice(id_0, size=n_class_1*100), id_1])
X_train_100_1 = X_train[id_subset]
y_train_100_1 = y_train[id_subset]

# 10:1
id_subset = np.concatenate([np.random.choice(id_0, size=n_class_1*10), id_1])
X_train_10_1 = X_train[id_subset]
y_train_10_1 = y_train[id_subset]

- **Logistic Regression - Undersampling**

**Data 100:1**

In [ ]:
cross_val_model(X_train_100_1, y_train_100_1, lr_model, n_splits=3, CM = True)

**Data 10:1**

In [ ]:
cross_val_model(X_train_10_1, y_train_10_1, lr_model, n_splits=3, CM = True)

- Os restultados são aparentemente ótimos!
Mas deve-se tomar muito cuidado pois muitos dados foram inutilizados, então é necessário fazer as validações na base de teste e tentar simular outros diferentes cenários para utilizar um modelo com undersampling.

### Criação do Modelos Finais (SKLearn X StatsModels)
Após serem selecionadas as variáveis, também pode ser realizado a tunagem de parametros por Grid Search, que nada mais é doq uma seleção dos parâmetros por força bruta (loop para testar diferentes valores e analisar o output). Além disso, também é importante decidir a métrica de erro a ser analisada, mas como são questões que dependem muito da base de dados em que se está trabalhando e do problema que queremos resolver (decisão de negócio), não vou explorar.

Além disso, nesse caso eu utilizei treino/validação e teste, somente treino e validação.
Novamente pelos motivos de ser apenas um exemplo, pois isso varia muito de acordo com os dados em questão. No caso real, pode se usar o último mês (ou qualquer período em que faça sentido para o negócio) para a base de teste, e todos os outros dados para treino e validação.

In [ ]:
# Variáveis finais após o Feature Selection
X = np.array(df.loc[:,np.array(var_x)[rfecv.support_]])
y = np.array(df.loc[:,var_y])

- **SKLearn**

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

pred = lr_model.predict(X_test)

In [ ]:
PlotCM(y_test, pred)

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

skplt.metrics.plot_roc_curve(y_test, lr_model.predict_proba(X_test))
plt.show()

skplt.metrics.plot_precision_recall_curve(y_test, lr_model.predict_proba(X_test))
plt.show()

- **StatsModels**
___
**Métodos:**
- logitreg.summary2()            # summary of the model
- logitreg.fittedvalues             # fitted value from the model
- logitreg.predict()                  # predict
- logfitreg.pred_table()           # confusion matrix

In [ ]:
import statsmodels.api as sm

lr_model = sm.Logit(y_train, X_train).fit()
pred_proba = lr_model.predict(X_test)
print(lr_model.summary2())

In [ ]:
# Odds ratio dos parametros
print("odds ratio")
print(np.exp(lr_model.params))

In [ ]:
# odds ratios and 95% CI
params = lr_model.params
conf = pd.DataFrame(lr_model.conf_int())
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print(np.exp(conf))

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, pred_proba)
roc_auc = auc(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)

#### Plota a curva ROC e linha 1-fpr
Também acha o valor ótimo para o threshold

In [ ]:
import pylab as pl
####################################
# The optimal cut off would be where tpr is high and fpr is low
# tpr - (1-fpr) is zero or near to zero is the optimal cut off point
####################################
i = np.arange(len(tpr)) # index for df
roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i), '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i), 'thresholds' : pd.Series(thresholds, index = i)})
print(roc.ix[(roc.tf-0).abs().argsort()[:1]])

# Plot tpr vs 1-fpr
fig, ax = pl.subplots()
pl.plot(roc['tpr'])
pl.plot(roc['1-fpr'], color = 'red')
pl.xlabel('1-False Positive Rate')
pl.ylabel('True Positive Rate')
pl.title('Receiver operating characteristic')
ax.set_xticklabels([])

#### Threshold

In [ ]:
# Função que encontra o valor ótimo do threshold
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value

    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold'])

In [ ]:
# Find optimal probability threshold
threshold = Find_Optimal_Cutoff(y_test, pred_proba)
print(threshold)

In [ ]:
def applyThresh(x, thresh):
    if x > thresh:
        return 1
    else:
        return 0

In [ ]:
# Acha as predicoes (0 e 1) aplicando o pred_proba
pred = np.vectorize(applyThresh)(pred_proba, threshold)

In [ ]:
PlotCM(y_test, pred)

Vemos que a Precision está muito baixa, mas podemos testar outros threshold, além de refinar o modelo analisando os outputs

In [ ]:
# Acha as predicoes (0 e 1) aplicando o pred_proba
pred = np.vectorize(applyThresh)(pred_proba, 0.6)

In [ ]:
PlotCM(y_test, pred)

### Cross Validation no StatsModels
Encontrei no SO esse "wraper" para usar o StatsModels com funcionalidades do SKLearn.
Aqui são somente alguns testes.

In [ ]:
import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin

class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

In [ ]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(SMWrapper(sm.Logit), X_train, y_train, scoring='neg_mean_squared_error'))

teste=cross_val_score(SMWrapper(sm.Logit), X_train, y_train, scoring='neg_mean_squared_error')

In [ ]:
teste

In [ ]:
lr_model = SMWrapper(sm.Logit)
lr_model.fit(X_train, y_train)

pred_proba = lr_model.predict(X_test)

In [ ]:
Find_Optimal_Cutoff(y_test, pred_proba)

In [ ]:
# Acha as predicoes (0 e 1) aplicando o pred_proba
pred = np.vectorize(applyThresh)(pred_proba, 0.0011455324286474824)
pred

In [ ]:
PlotCM(y_test, pred)